In [418]:
#Library
import pandas as pd 
from pmaw import PushshiftAPI
import datetime
import requests
import json
import psaw
import spacy

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from collections import Counter
import flair
import ast

In [392]:
api = PushshiftAPI()

In [419]:
# before = int(datetime.datetime(2021,1,1,0,0).timestamp()) 
# start_time =  int(datetime.datetime(2021,3,6,0,0).timestamp())

before = int(datetime.datetime(2021,3,16,0,0).timestamp())
after = int(datetime.datetime(2021,1,1,0,0).timestamp())

submissions = api.search_submissions(before=before,
                                     after=after,
                                     subreddit='stocks',
                                     filter=['url','subreddit',  'author', 'title','removed_by_category',
                                             'selftext', 'upvote_ratio', 'ups', 'down', 'score'])
                                     #stop_condition=lambda x: 'moderator' in x.removed_by_category,
                                     #limit=1000)

submissions = pd.DataFrame(submissions)

42276 results available in Pushshift
Checkpoint:: Success Rate: 100.00% - Requests: 100 - Batches: 10 - Items Remaining: 32294
Checkpoint:: Success Rate: 100.00% - Requests: 200 - Batches: 20 - Items Remaining: 23297
Checkpoint:: Success Rate: 100.00% - Requests: 300 - Batches: 30 - Items Remaining: 14533
Checkpoint:: Success Rate: 100.00% - Requests: 400 - Batches: 40 - Items Remaining: 6300
Checkpoint:: Success Rate: 100.00% - Requests: 500 - Batches: 50 - Items Remaining: 97
Total:: Success Rate: 100.00% - Requests: 505 - Batches: 51 - Items Remaining: 3
Total:: Success Rate: 100.00% - Requests: 506 - Batches: 52 - Items Remaining: 0


In [428]:
submissions.to_csv('data/reddit_stocks_2021')

In [431]:
reddit_stocks = pd.read_csv('data/reddit_stocks_2021')


reddit_stocks = reddit_stocks.sort_values(by=['created_utc'], ascending=True)
reddit_stocks.reset_index(drop=True, inplace=True)
#transform epoch time to date format
reddit_stocks['date'] = reddit_stocks['created_utc'].map(lambda val: datetime.datetime.fromtimestamp(val).strftime('%Y-%m-%d'))
reddit_stocks['date'] = pd.to_datetime(reddit_stocks['date'])
#combine title and selftext together
reddit_stocks['post'] = reddit_stocks['title'] + reddit_stocks['selftext'].astype(str)

,Unnamed: 0,author,created_utc,score,selftext,subreddit,title,upvote_ratio,url,removed_by_category
42271,904,Bob__Kazamakis,1615865386,1,[removed],stocks,The stock market don’t make no sense to me,1.0,https://www.reddit.com/r/stocks/comments/m5zzd...,moderator
42272,903,t-carter41,1615865543,1,I saw this post the other day knocking down Mo...,stocks,Any recs on stocks information and research?,1.0,https://www.reddit.com/r/stocks/comments/m6011...,NaN
42273,902,ImNotSmelly,1615865946,1,I would have to believe the CCP runs a skim on...,stocks,Would anyone be surprised if these Chinese sto...,1.0,https://www.reddit.com/r/stocks/comments/m6058...,NaN
42274,901,JollyGreeneGiants,1615866088,1,"KE Holdings, a leading integrated online and o...",stocks,KE Holdings [BEKE] posts strong fourth quarter...,1.0,https://www.reddit.com/r/stocks/comments/m606o...,NaN
42275,900,wavefool,1615866588,1,[removed],stocks,Debit Spread help,1.0,https://www.reddit.com/r/stocks/comments/m60bu...,moderator


In [439]:
reddit_stocks.head()

,Unnamed: 0,author,created_utc,score,selftext,subreddit,title,upvote_ratio,url,removed_by_category,date,post
0,15455,izgustant,1609477274,1,"Dear future stonk millionaires,\n\nFirst of al...",stocks,Question about the 2021 stocks and your opinio...,1.0,https://www.reddit.com/r/stocks/comments/ko5e2...,NaN,2021-01-01,Question about the 2021 stocks and your opinio...
1,15454,Andymilonakis21,1609478082,1,[removed],stocks,So I hear a lot of people say don’t do options...,1.0,https://www.reddit.com/r/stocks/comments/ko5kz...,moderator,2021-01-01,So I hear a lot of people say don’t do options...
2,7799,PuzzledMindAndBody,1609478237,1,[removed],stocks,Index Funds:Stocks Allocation,1.0,https://www.reddit.com/r/stocks/comments/ko5md...,moderator,2021-01-01,Index Funds:Stocks Allocation[removed]
3,7798,bigbang1996,1609478251,1,"According to a JD power survey, Tesla was rank...",stocks,High confidence - Nio,1.0,https://www.reddit.com/r/stocks/comments/ko5mh...,NaN,2021-01-01,High confidence - NioAccording to a JD power s...
4,7797,PuzzledMindAndBody,1609478426,1,[removed],stocks,Index Funds:Stocks Allocation %,1.0,https://www.reddit.com/r/stocks/comments/ko5o1...,moderator,2021-01-01,Index Funds:Stocks Allocation %[removed]


In [440]:
nlp = spacy.load('en_core_web_sm')

In [441]:
# remove specific term 
BLACKLIST = ['Vanguard','Newbie', 'YouTube','EV','Fidelity', 
             "McDonald’s", 'Boeing', 'Blackrock', 'Merrill Lynch', 
             'TransferWise', 'Renters/Home', 'Lemonade','RENT &', "removed"]

def get_orgs(text):
    doc = nlp(text)
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list



In [442]:
#find entity among posts
reddit_stocks['orgs'] = reddit_stocks['post'].apply(get_orgs)

In [446]:
reddit_stocks.head()

,Unnamed: 0,author,created_utc,score,selftext,subreddit,title,upvote_ratio,url,removed_by_category,date,post,orgs
0,15455,izgustant,1609477274,1,"Dear future stonk millionaires,\n\nFirst of al...",stocks,Question about the 2021 stocks and your opinio...,1.0,https://www.reddit.com/r/stocks/comments/ko5e2...,NaN,2021-01-01,Question about the 2021 stocks and your opinio...,"[IRL, NASDAQ]"
1,15454,Andymilonakis21,1609478082,1,[removed],stocks,So I hear a lot of people say don’t do options...,1.0,https://www.reddit.com/r/stocks/comments/ko5kz...,moderator,2021-01-01,So I hear a lot of people say don’t do options...,[]
2,7799,PuzzledMindAndBody,1609478237,1,[removed],stocks,Index Funds:Stocks Allocation,1.0,https://www.reddit.com/r/stocks/comments/ko5md...,moderator,2021-01-01,Index Funds:Stocks Allocation[removed],[]
3,7798,bigbang1996,1609478251,1,"According to a JD power survey, Tesla was rank...",stocks,High confidence - Nio,1.0,https://www.reddit.com/r/stocks/comments/ko5mh...,NaN,2021-01-01,High confidence - NioAccording to a JD power s...,[]
4,7797,PuzzledMindAndBody,1609478426,1,[removed],stocks,Index Funds:Stocks Allocation %,1.0,https://www.reddit.com/r/stocks/comments/ko5o1...,moderator,2021-01-01,Index Funds:Stocks Allocation %[removed],[]


In [447]:
reddit_stocks = reddit_stocks[['date','author', 'post', 'orgs']]
reddit_stocks.head(30)

,date,author,post,orgs
0,2021-01-01,izgustant,Question about the 2021 stocks and your opinio...,"[IRL, NASDAQ]"
1,2021-01-01,Andymilonakis21,So I hear a lot of people say don’t do options...,[]
2,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation[removed],[]
3,2021-01-01,bigbang1996,High confidence - NioAccording to a JD power s...,[]
4,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation %[removed],[]
5,2021-01-01,PuzzledMindAndBody,Index funds: StocksI am 36M. Couple of questio...,[]
6,2021-01-01,WalkerTejasRanger,Compounding interest questionNew to investing ...,[]
7,2021-01-01,Kyle_Ace_22,"Top Stock Picks For 2021?What stocks, etf's, o...","[AAPL, ICLN, AMZN, IWF]"
8,2021-01-01,sco77001,January 2021 stock market performance?[removed],[]
9,2021-01-01,Substantial_Net4379,BNGO[removed],[]


In [448]:
reddit_stocks.to_csv('data/reddit_stocks')

In [703]:
reddit_stocks.head()

,date,author,post,orgs,sentences,scores,scores_sum
0,2021-01-01,izgustant,Question about the 2021 stocks and your opinio...,"[IRL, NASDAQ]",[Question about the 2021 stocks and your opini...,"[0.985, 0.997, 0.983, -0.978, -0.963, -0.879, ...",-2.385
1,2021-01-01,Andymilonakis21,So I hear a lot of people say don’t do options...,[],[So I hear a lot of people say don’t do option...,[-0.973],-0.973
2,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation[removed],[],[Index Funds:Stocks Allocation[removed]],[-0.951],-0.951
3,2021-01-01,bigbang1996,High confidence - NioAccording to a JD power s...,[],[High confidence - NioAccording to a JD power ...,"[0.999, 0.997]",1.996
4,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation %[removed],[],[Index Funds:Stocks Allocation %[removed]],[-0.994],-0.994


In [449]:
# merge organizations column into one big list
orgs = reddit_stocks['orgs'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['IRL',
 'NASDAQ',
 'AAPL',
 'ICLN',
 'AMZN',
 'IWF',
 'HS',
 'FOREX',
 'SEC',
 'insider_trading_program/g9cmzvi/?context=3']

In [619]:
print(org_freq)

Counter({'GME': 2044, 'AMC': 857, 'EV': 396, 'Apple': 370, 'WSB': 353, 'Amazon': 290, 'PLTR': 268, 'NOK': 259, 'Tesla': 238, 'AAPL': 209, 'Fidelity': 202, 'SEC': 199, 'AMD': 197, 'ICLN': 192, 'ARK': 189, '🚀': 162, 'Microsoft': 160, 'ARKK': 156, 'FDA': 150, 'GameStop': 150, 'NYSE': 149, 'NASDAQ': 142, 'Nokia': 141, 'EPS': 115, 'APHA': 114, 'Ford': 106, 'Newbie': 105, 'NIO': 101, 'NAKD': 97, 'GM': 96, 'Robinhood': 91, 'YouTube': 89, 'ARKG': 88, 'Disney': 88, 'Nasdaq': 84, 'CNBC': 82, 'GE': 81, 'EU': 80, 'Vanguard': 78, 'AI': 76, 'AMZN': 71, 'Boeing': 71, 'ITM': 69, 'ATH': 67, 'Reuters': 66, 'Palantir': 65, 'BB': 63, 'MSFT': 62, 'Tilray': 61, 'SPY': 60, 'FOMO': 60, 'RSI': 59, 'Intel': 58, 'Treasury': 57, 'Google': 57, 'CCIV': 57, 'PE': 56, 'ETFs': 56, 'VOO': 55, 'Senate': 55, 'SLV': 51, 'Fed': 50, 'Cathie Wood': 48, 'CRSR': 46, 'Fundamentals](https://www.reddit.com': 45, 'Trading](https://www.reddit.com': 45, 'DIS': 45, 'Congress': 44, 'NET': 44, 'YoY': 44, '’ve': 43, 'Morgan Stanley': 43

In [615]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)
org_freq.most_common(25)

[('GME', 2044),
 ('AMC', 857),
 ('EV', 396),
 ('Apple', 370),
 ('WSB', 353),
 ('Amazon', 290),
 ('PLTR', 268),
 ('NOK', 259),
 ('Tesla', 238),
 ('AAPL', 209),
 ('Fidelity', 202),
 ('SEC', 199),
 ('AMD', 197),
 ('ICLN', 192),
 ('ARK', 189),
 ('🚀', 162),
 ('Microsoft', 160),
 ('ARKK', 156),
 ('FDA', 150),
 ('GameStop', 150),
 ('NYSE', 149),
 ('NASDAQ', 142),
 ('Nokia', 141),
 ('EPS', 115),
 ('APHA', 114)]

In [647]:
df_90 = pd.DataFrame([org_freq])
df_90 = df_90.T
df_90.columns = ['frequency']
df_90 = df_90.sort_values(by='frequency', ascending=False)
df_orgs_frequency = df_90.head(20)
df_orgs_frequency.to_csv('data/orgs_frequency')

In [451]:
from flair.data import Sentence
from segtok.segmenter import split_single
import re

In [452]:
classifier = flair.models.TextClassifier.load('en-sentiment')

2021-03-16 14:36:25,115 loading file C:\Users\Yuri\.flair\models\sentiment-en-mix-distillbert_4.pt


In [453]:
def make_sentences(text):
    """ Break apart text into a list of sentences """
    sentences = [sent for sent in split_single(text)]
    return sentences

In [454]:
def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

In [455]:
def predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
    # stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
    else:
        result = -(text.to_dict()['labels'][0]['confidence'])
    return round(result, 3)

def get_scores(sentences):
    """ Call predict on every sentence of a text """
    results = []
    
    for i in range(0, len(sentences)): 
        results.append(predict(sentences[i]))
    return results

def get_sum(scores):
    
    result = round(sum(scores), 3)
    return result

In [456]:
#transform the post into sentence and get the sentiment analysis score
reddit_stocks.post = reddit_stocks.post.apply(clean)
reddit_stocks['sentences'] = reddit_stocks.post.apply(make_sentences)
reddit_stocks['scores'] = reddit_stocks['sentences'].apply(get_scores)
reddit_stocks['scores_sum'] = reddit_stocks.scores.apply(get_sum)

reddit_stocks.head(20)

,date,author,post,orgs,sentences,scores,scores_sum
0,2021-01-01,izgustant,Question about the 2021 stocks and your opinio...,"[IRL, NASDAQ]",[Question about the 2021 stocks and your opini...,"[0.985, 0.997, 0.983, -0.978, -0.963, -0.879, ...",-2.385
1,2021-01-01,Andymilonakis21,So I hear a lot of people say don’t do options...,[],[So I hear a lot of people say don’t do option...,[-0.973],-0.973
2,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation[removed],[],[Index Funds:Stocks Allocation[removed]],[-0.951],-0.951
3,2021-01-01,bigbang1996,High confidence - NioAccording to a JD power s...,[],[High confidence - NioAccording to a JD power ...,"[0.999, 0.997]",1.996
4,2021-01-01,PuzzledMindAndBody,Index Funds:Stocks Allocation %[removed],[],[Index Funds:Stocks Allocation %[removed]],[-0.994],-0.994
5,2021-01-01,PuzzledMindAndBody,Index funds: StocksI am 36M. Couple of questio...,[],"[Index funds: StocksI am 36M., Couple of quest...","[-0.971, 0.74, -0.997, 0.848]",-0.380
6,2021-01-01,WalkerTejasRanger,Compounding interest questionNew to investing ...,[],[Compounding interest questionNew to investing...,"[-0.999, -0.999, -0.999, -0.793, -0.999, -0.84...",-4.641
7,2021-01-01,Kyle_Ace_22,"Top Stock Picks For 2021?What stocks, etf's, o...","[AAPL, ICLN, AMZN, IWF]","[Top Stock Picks For 2021?What stocks, etf's, ...","[0.997, 0.974, 0.963]",2.934
8,2021-01-01,sco77001,January 2021 stock market performance?[removed],[],[January 2021 stock market performance?[removed]],[-0.994],-0.994
9,2021-01-01,Substantial_Net4379,BNGO[removed],[],[BNGO[removed]],[-0.977],-0.977


In [457]:
reddit_stocks.tail(20)

,date,author,post,orgs,sentences,scores,scores_sum
42256,2021-03-15,JoacoFerna24,$DNN due diligence - Helpful info on company a...,[],[$DNN due diligence - Helpful info on company ...,[-1.0],-1.000
42257,2021-03-15,Panditji15,PLTR bullish signal and trade idea[removed],[PLTR],[PLTR bullish signal and trade idea[removed]],[-0.996],-0.996
42258,2021-03-15,inkslingerben,SEC Sues California trader for alleged social ...,"[SEC, The U.S. Securities and Exchange Commiss...",[SEC Sues California trader for alleged social...,"[-0.998, -0.761, 0.913, -0.929]",-1.775
42259,2021-03-15,StaggerHiramLee,I’m new. Why does it seem like being a pattern...,[],"[I’m new., Why does it seem like being a patte...","[0.999, -0.986]",0.013
42260,2021-03-15,JoacoFerna24,$DNN Due Diligence - Helpful info on the compa...,[],[$DNN Due Diligence - Helpful info on the comp...,[-0.999],-0.999
42261,2021-03-15,BenniHB,help with finance website?[removed],[],[help with finance website?[removed]],[-0.991],-0.991
42262,2021-03-15,drewgarr,What’s your opinion on Shaw Stock priceWith th...,"[Shaw, Shaw Stock, Shaw I, Rogers]",[What’s your opinion on Shaw Stock priceWith t...,"[0.948, 0.867, -0.944]",0.871
42263,2021-03-15,StatisticianLoud7762,CTRM IS GOING TO BLOW TOMMOROW[removed],[],[CTRM IS GOING TO BLOW TOMMOROW[removed]],[-1.0],-1.000
42264,2021-03-15,No-Consequence-8713,NIO buy buy buy buy[removed],[],[NIO buy buy buy buy[removed]],[-0.999],-0.999
42265,2021-03-15,Scary_Marionberry140,EXPR BUY NOW[removed],[],[EXPR BUY NOW[removed]],[-0.858],-0.858


In [464]:
reddit_copy = reddit_stocks.explode('orgs')

reddit_copy.reset_index(drop=True, inplace=True)
reddit_copy = reddit_copy.dropna()


In [545]:
reddit_copy.to_csv('data/reddit_copy')

In [544]:
#reddit_copy.reset_index(drop=True, inplace=True)
reddit_copy.head()
#reddit_copy['orgs'].isna().sum()

,date,author,post,orgs,sentences,scores,scores_sum
0,2021-01-01,izgustant,Question about the 2021 stocks and your opinio...,IRL,[Question about the 2021 stocks and your opini...,"[0.985, 0.997, 0.983, -0.978, -0.963, -0.879, ...",-2.385
1,2021-01-01,izgustant,Question about the 2021 stocks and your opinio...,NASDAQ,[Question about the 2021 stocks and your opini...,"[0.985, 0.997, 0.983, -0.978, -0.963, -0.879, ...",-2.385
2,2021-01-01,Kyle_Ace_22,"Top Stock Picks For 2021?What stocks, etf's, o...",AAPL,"[Top Stock Picks For 2021?What stocks, etf's, ...","[0.997, 0.974, 0.963]",2.934
3,2021-01-01,Kyle_Ace_22,"Top Stock Picks For 2021?What stocks, etf's, o...",ICLN,"[Top Stock Picks For 2021?What stocks, etf's, ...","[0.997, 0.974, 0.963]",2.934
4,2021-01-01,Kyle_Ace_22,"Top Stock Picks For 2021?What stocks, etf's, o...",AMZN,"[Top Stock Picks For 2021?What stocks, etf's, ...","[0.997, 0.974, 0.963]",2.934


In [603]:
spy_df = pd.DataFrame(reddit_copy.groupby(["date"], as_index=False).agg({'scores_sum':'mean'}))
spy_df.head(20)

,date,scores_sum
0,2021-01-01,0.780687
1,2021-01-02,-17.502110
2,2021-01-03,-2.139727
3,2021-01-04,2.398160
4,2021-01-05,-4.835623
5,2021-01-06,-2.440988
6,2021-01-07,-4.440185
7,2021-01-08,-3.209271
8,2021-01-09,-18.900658
9,2021-01-10,3.550024


In [605]:
spy_df.columns = ['date', 'sentiment_score']
spy_df.head(20)

,date,sentiment_score
0,2021-01-01,0.780687
1,2021-01-02,-17.502110
2,2021-01-03,-2.139727
3,2021-01-04,2.398160
4,2021-01-05,-4.835623
5,2021-01-06,-2.440988
6,2021-01-07,-4.440185
7,2021-01-08,-3.209271
8,2021-01-09,-18.900658
9,2021-01-10,3.550024


In [610]:
import ffn #for loading financial data

## merge stock price and plot

spy=ffn.get('spy', start='2021-01-01')
spy_vals=[]
for date in spy_df['date'].tolist():
    try:
        spy_vals.append(float(spy.loc[date]))
    except KeyError:
        spy_vals.append(None)  
spy_df['spy']=spy_vals

spy_df=spy_df[['date','sentiment_score','spy']]
spy_df=spy_df.set_index('date')
spy_df=spy_df[df_spy['spy'].notna()]

spy_df.to_csv('data/positive+negative_sentiment_data.csv')

KeyError: 'date'

In [695]:
AMC_sentiments = reddit_copy[reddit_copy['orgs'] == 'AMC']

In [698]:
AMC_sentiments = pd.DataFrame(AMC_sentiments.groupby(["date", "orgs"], as_index=False)["scores_sum"].mean())

AMC_sentiments.columns = ['date','orgs', 'sentiment_score']

AMC_sentiments.head()

,date,orgs,sentiment_score
0,2021-01-04,AMC,-4.7360
1,2021-01-05,AMC,-2.0960
2,2021-01-07,AMC,-2.7360
3,2021-01-08,AMC,-1.6270
4,2021-01-15,AMC,-0.1025


In [700]:
## merge stock price and plot for apple

AMC=ffn.get('AMC', start='2021-01-01')
AMC_vals=[]
for date in AMC_sentiments['date'].tolist():
    try:
        AMC_vals.append(float(AMC.loc[date]))
    except KeyError:
        AMC_vals.append(None)  
AMC_sentiments['AMC']=AMC_vals

AMC_sentiments=AMC_sentiments[['date','sentiment_score','AMC']]
AMC_sentiments=AMC_sentiments.set_index('date')
AMC_sentiments=AMC_sentiments[AMC_sentiments['AMC'].notna()]

#spy_df.to_csv('data/positive+negative_sentiment_data.csv')

In [701]:
AMC_sentiments.head()

,sentiment_score,AMC
date,,
2021-01-04,-4.7360,2.01
2021-01-05,-2.0960,1.98
2021-01-07,-2.7360,2.05
2021-01-08,-1.6270,2.14
2021-01-15,-0.1025,2.33


In [702]:
AMC_sentiments.to_csv('data/amc_sentiments.csv')

In [541]:

#rolling average 
test_14['14day'] = test_14.scores_sum.rolling(14).mean().shift(-7)

In [480]:
import plotly.express as px
import plotly.graph_objects as go

In [536]:
fig = go.Figure([go.Scatter(x=test_14['date'], y=test_14['scores_sum'])])
fig.show()